### Cloud Kubernetes Services<br>

***

<br>

Services are major networking component when it comes to working in Kubernetes<br>
Can play a major factor when it comes to deciding how you want to route your traffic within your Kubernetes cluster<br>

**Kubernetes approach to networking**<br>

Kubernetes pods are ephemeral. Pods are created and destroyed to match the state of your clusters<br>
pods in deployment mode gets new IP and can't track these, as they are very frequent<br>
so this is where services come into play<br>
<br>

**What's a Service?**<br>

Service is an abstraction in a process, it's not a process that listens on some network interface<br>
service can be defined as a logical set of pods, an abstract on top of the pod<br>

- Persistent IP<br>
which provides single persistent IP and a DNS name, which pods can be accessed<br>

- Internal and External<br>
It allows routing external traffic into your Kubernetes cluster and used inside of your cluster for intelligent routing<br>

- Load balancing<br>
Easy to manage load balancing configuration for traffic between replicas<br>

- Scaling<br>
Helps pods scales quickly and easily, as the service will automatically handle the recreation of pods and their new IP<br>

**Main goal of Service in Kubernetes is to provide persistent access to its pods without necessity to look for pods IP each time when pod is recreated**<br>
Services allow external access from users to the application inside the cluster without having to know the IP of individual pod<br>

**Service Components:**<br>

In [ ]:
# services.yaml (Service Manifests)

apiVersion: v1
kind: Service
metadata:
    name: clusterip-service # DNS name of the service
spec:
  selector:
    app: inventory # Forward requests to pods with this label
  type: ClusterIP # type of service you'd want to use
  ports:
  - protocol: TCP
    port: 80 # port number exposed internally in cluster
    targetPort: 80 # port that containers are listening on

# deployment.yaml (Deployment Manifests)

apiVersion: apps/v1
kind: Deployment
metadata:
    name: service1
spec:
  replicas: 2
  template:
    metadata:
      labels:
        app: inventory
  spec:
    containers:
    - name: service
      image: nginx:latest

**Selectors and Labels**<br>

User -> Service (Selector: app: inventory) -> Node1(pod1: nginx, labels: app: inventory), Node2(pod2: nginx, labels: app: shop)

**Service select pods based on their labels**<br>
when a selector request is made to the service<br>
it selects all pods in the cluster, matching the key/value pair under the selector<br>
it forwards the network request to it<br>
<br>

##### **Service Types:**<br>

Kubernetes Services provides the interfaces through which pods can communicate with each other<br>
They also act as a main gateway for application, services use selectors to identify which pods they should control<br>
They expose an IP address and ports, services can expose more than 1 port, can also route traffics to other services<br>
External IP or DNS names<br>

Services make it easy to create network services in Kubernetes<br>
each service can be backed with many pods as needed<br> 
without having to make your code aware of how each services are backed<br>
<br>

- **ClusterIP**<br>

default kubernetes service<br>
service is not exposed outside of the cluster, but can be addressed within the cluster<br>
kubernetes create a stable IP that is accessible from nodes in the cluster<br>
clients in the cluster call the service by using the cluster IP and port value<br>
the request is forwarded to the target-port<br>

User -> Service (10.176.133.7 TCP Port: 80) -> Node1(Pod1 targetPort80), Node2(Pod2 targetPort80)<br>
<br>

- **NodePort**<br>

you specify nodePort value<br>
nodePort is a static port, it's chosen from pre-configured range(30000-32767)<br>
User can access the node via Node IP + nodePort value.<br>

This method isn't very secure, as it opens each node to external entry.<br>
and relies on knowing the IP of each node, which could change anytime<br>
<br>

- **LoadBalancer**<br>

This service is exposed as a load balancer in the cluster<br>
loadbalancer services will create an internal kubernetes services connected a cloud provider's loadbalancer<br>
in this case: Google Cloud<br>
this will create a static, publicly addressable IP and a DNS name that can be used to access the cluster from an external source<br>

loadbalancer type is an extension of nodeport type.<br>
service of type loadbalancer naturally has a cluster IP<br>
if you want to directly expose a service, this is the default method<br>
you can send many types of traffic to it, like: HTTP(S), TCP, UDP and more<br>

downside is each service you expose with a loadbalancer, you pay for loadbalancer<br>
it can rack up the bill if you're using multiple loadbalancer<br>
<br>

- **Ingress**<br>

Skipped..
<br>

- **Multi-port Services**<br>

for some services, there need to expose more than 1 port.<br>
kubernetes let you configure multiple port definition on a service object<br>
when using multiple port for a service, you must give all your port names<br>
if you've multiple service ports, these names must be unique<br>

port 80 -> forwarded to both node1, node2<br>
port 9752 -> forwarded only to node1<br>

each pod must have a container listening to tcp port<br>
this could be a single container with 2 threads or 2 containers running in the same pod<br>
<br>

- **ExternalName**<br>

provides an internal alias for an external DNS name<br>
internal clients makes request using internal DNS name, requests are redirected to external name<br>

this is mapping internal DNS name to external DNS name<br>
great use for any external service that resides outside of your cluster<br>

name: sql1
externalName: sql1.inc.private
<br>

- **Headless**<br>

you can create a headless service by specifying none as a service type in the manifest file<br>
this option also allows you to choose other service discovery mechanisms without being tied Kubernetes implementation<br>
applications can still use self registration pattern with this service<br>

great use case: when you don't need a loadbalancer or routing, only need the service to patch the request to the backend pod, no IPs needed<br>
headless is typically used with StatefulSets, where the name of the pods are fixed.<br>
useful when you're setting up a MySQL cluster or need to know the name of the master<br>
<br>